In [5]:
import os
from pymongo import MongoClient
import pandas as pd

# --- Config ---
MONGO_URI = os.getenv("MONGODB_URI", "mongodb://localhost:27017")
DB_NAME = "wowaudit_database"
COLL_NAME = "wowaudit_hourly"
PLAYER_NAME = "Nyph"

# --- Connect ---
client = MongoClient(MONGO_URI)
coll = client[DB_NAME][COLL_NAME]

# --- Query: only what we need, for Nyph ---
pipeline = [
    {"$match": {"metadata.name": PLAYER_NAME}},
    {"$project": {
        "_id": 0,
        "timestamp": 1,
        "season_mythic_dungeons": 1,
        "metadata.expansion": 1,
        "metadata.season": 1
    }},
    {"$sort": {"timestamp": 1}}
]

rows = list(coll.aggregate(pipeline))

# --- Frame it ---
if not rows:
    raise SystemExit(f"No rows found for player {PLAYER_NAME}")

df = pd.DataFrame(rows)

# Flatten metadata fields
df["expansion"] = df["metadata"].apply(lambda m: m.get("expansion") if isinstance(m, dict) else None)
df["season"]    = df["metadata"].apply(lambda m: m.get("season") if isinstance(m, dict) else None)
df = df.drop(columns=["metadata"])

# Ensure timestamp is timezone-aware UTC (Sheets likes ISO strings)
df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)

# Helpful combined label if you want it in Sheets
df["expansion_season"] = df.apply(
    lambda r: f'{r.get("expansion", "")} {str(r.get("season", ""))}'.strip(),
    axis=1
)

# Optional: per-season "delta" of season_mythic_dungeons
# If season_mythic_dungeons is a running season total (typical), delta shows new dungeons since last sample.
df = df.sort_values(["season", "timestamp"])
df["delta"] = (
    df.groupby("season")["season_mythic_dungeons"]
      .diff()
      .fillna(0)
      .clip(lower=0)
      .astype(int)
)

# Final tidy columns for Google Sheets
out = df[[
    "timestamp",
    "expansion",
    "season",
    "expansion_season",
    "season_mythic_dungeons",
    "delta"
]]

# If you want local CSV to import to Sheets:
out.to_csv("nyph_mplus_by_season.csv", index=False)
print("Wrote nyph_mplus_by_season.csv")

# --- Tips for plotting in Google Sheets ---
# 1) Import the CSV into a sheet.
# 2) Insert → Chart:
#    - For cumulative lines over time: use "timestamp" as X-axis, add a series for "season_mythic_dungeons".
#    - If you want separate lines per season in one chart:
#         * Create a pivot table with Rows = timestamp, Columns = season,
#           Values = MAX of season_mythic_dungeons.
#         * Chart the pivot table as a line chart (one line per season).
#    - For bars of "new dungeons per sample": use the "delta" series instead.


Wrote nyph_mplus_by_season.csv


In [ ]:
{
  "timestamp": {
    "$date": "2024-09-08T20:29:58.000Z"
  },
  "metadata": {
    "expansion": "TWW",
    "guild": "Eternal Kingdom",
    "name": "Boltari",
    "realm": "Proudmoore",
    "season": "S1",
    "team_name": "Quartz",
    "team_realm": "Proudmoore",
    "team_region": "US"
  },
  "best_waist_id": 224672,
  "head_id": 221187,
  "ilvl": 579.81,
  "chest_id": 224666,
  "tier_shoulder_ilvl": 0,
  "best_finger_2_id": 221136,
  "trinket_1_id": 219295,
  "WCL_normal": "-|-|-|-|-|-|-|-",
  "neck_id": 221060,
  "historical_great_vault_slot_6": "-",
  "finger_2_enchant": "Glimmering Haste (1)",
  "epic_gem": 0,
  "primordial_stone_2_name": 0,
  "upgrade_level_head": "1 / 8",
  "siege_of_boralus_score": 0,
  "best_finger_1_id": 215130,
  "_id": {
    "$oid": "66f85478bdaef808e2e73d1e"
  },
  "raids_visible": "yes",
  "raids_raid_finder_weekly": "0|0|0|0|0|0|0|0",
  "raids_normal_weekly": "0|0|0|0|0|0|0|0",
  "rbg_week_played": 0,
  "primordial_stone_3_name": 0,
  "runed_crests": 0,
  "best_waist_quality": 3,
  "titles": 103,
  "best_trinket_1_ilvl": 580,
  "best_trinket_1_name": "Sigil of Algari Concordance",
  "neck_name": "Strapped Rescue-Keg",
  "best_head_id": 221187,
  "best_shoulder_name": "Alighted Shoulderpads of the Peerless",
  "waist_quality": 3,
  "upgrade_level_hands": "4 / 8",
  "weathered_crests": 300,
  "hands_ilvl": 580,
  "professions_visible": "yes",
  "legs_id": 221050,
  "the_stonevault_score": 0,
  "historical_great_vault_slot_9": "-",
  "shoulder_id": 224671,
  "awakening_the_machine": "yes",
  "ahead_of_the_curve": 22,
  "lvl_25_pets": 28,
  "tier_chest_ilvl": 0,
  "upgrade_level_main_hand": "4 / 8",
  "historical_dungeons_done": "0",
  "main_hand_name": "Melodious Iron Songspire",
  "achievement_points": 24545,
  "shuffle_rating": 0,
  "season_heroic_dungeons": 28,
  "best_head_quality": 3,
  "best_feet_ilvl": 580,
  "wrist_id": 222833,
  "shoulder_ilvl": 571,
  "upgrade_level_finger_2": "1 / 8",
  "trinket_1_name": "Sigil of Algari Concordance",
  "historical_wqs_done": "0",
  "profession_1": "Enchanting (76)",
  "enchant_quality_main_hand": 0,
  "arakara_city_of_echoes_score": 0,
  "best_trinket_1_id": 219295,
  "upgrade_level_neck": "4 / 8",
  "trinket_2_ilvl": 593,
  "best_hands_id": 221067,
  "character_id": 3728363,
  "quests_done_total": 14789,
  "assembly_of_the_deeps_renown": 6,
  "faction": "Alliance",
  "best_finger_1_ilvl": 590,
  "best_trinket_2_ilvl": 593,
  "hands_quality": 3,
  "waist_ilvl": 580,
  "best_head_name": "Intertwiner's Shadowcowl",
  "main_hand_ilvl": 580,
  "finger_2_id": 221136,
  "head_quality": 3,
  "upgrade_level_chest": "4 / 8",
  "best_trinket_2_id": 219314,
  "WCL_mythic": "-|-|-|-|-|-|-|-",
  "the_necrotic_wake_total": 0,
  "best_legs_ilvl": 580,
  "chest_name": "Alighted Robes of the Fireflash",
  "the_necrotic_wake_score": 0,
  "best_trinket_2_quality": 4,
  "enchant_quality_wrist": 0,
  "wqs_this_week": 31,
  "best_off_hand_id": 0,
  "ingenuity_sparks_equipped": 0,
  "severed_threads_renown": 5,
  "max_3v3_rating": 1500,
  "unique_pets": 462,
  "spark_off_hand_ilvl": 0,
  "shuffle_week_played": 0,
  "trinket_2_name": "Ara-Kara Sacbrood",
  "dungeons_this_week": 0,
  "enchant_quality_chest": 0,
  "enchant_quality_finger_1": 0,
  "3v3_rating": 0,
  "shoulder_name": "Alighted Shoulderpads of the Peerless",
  "gender": "Male",
  "enchant_quality_head": 0,
  "mounts": 317,
  "race": "Night Elf",
  "best_hands_quality": 3,
  "best_hands_name": "Punctured Apiary Gloves",
  "best_shoulder_id": 224671,
  "mists_of_tirna_scithe_score": 0,
  "mists_of_tirna_scithe_total": 0,
  "best_wrist_ilvl": 590,
  "spark_off_hand_quality": 0,
  "main_hand_id": 221085,
  "council_of_dornogal_renown": 7,
  "best_legs_id": 221050,
  "best_off_hand_quality": 0,
  "best_back_name": "Gem-Woven Cloak of the Peerless",
  "back_name": "Gem-Woven Cloak of the Peerless",
  "shuffle_season_played": 0,
  "toys_owned": 370,
  "raids_mythic_weekly": "0|0|0|0|0|0|0|0",
  "exalted_amount": 92,
  "siege_of_boralus_total": 0,
  "best_hands_ilvl": 580,
  "feet_id": 221043,
  "city_of_threads_score": 0,
  "2v2_season_played": 0,
  "best_wrist_id": 222833,
  "3v3_week_played": 0,
  "the_dawnbreaker_score": 0,
  "best_head_ilvl": 571,
  "2v2_week_played": 0,
  "theater_troupe": "yes",
  "arakara_city_of_echoes_total": 0,
  "tier_legs_ilvl": 0,
  "enchant_quality_back": 0,
  "coffer_keys": 9,
  "best_feet_id": 221043,
  "waist_enchant": 0,
  "upgrade_level_trinket_2": "8 / 8",
  "rbg_season_played": 0,
  "raids_normal": "0|0|0|0|0|0|0|0",
  "best_main_hand_name": "Melodious Iron Songspire",
  "the_stonevault_total": 0,
  "upgrade_level_finger_1": "-",
  "realm": "Proudmoore",
  "historical_great_vault_slot_3": "-",
  "best_waist_name": "Alighted Cord of the Feverflare",
  "historical_great_vault_slot_1": "-",
  "finger_2_ilvl": 571,
  "best_legs_name": "Ancient Hardened Legwraps",
  "worldsoul_memories": 1,
  "legs_ilvl": 580,
  "best_finger_1_quality": 4,
  "wrist_name": "Pioneer's Cloth Cuffs",
  "name": "Boltari",
  "wqs_done_total": 14678,
  "honor_level": 13,
  "m+_score": 0,
  "week_delves": 2,
  "neck_quality": 3,
  "trinket_2_id": 219314,
  "best_shoulder_quality": 3,
  "best_finger_1_name": "Malleable Band",
  "best_off_hand_name": 0,
  "waist_id": 224672,
  "feet_name": "Cloudstrider Soles",
  "upgrade_level_trinket_1": "4 / 8",
  "total_upgrades_missing": 60,
  "weekly_highest_m+": 0,
  "primordial_stone_1_name": 0,
  "join_date": "2024-09-04 04:53:26 +0000",
  "best_finger_2_ilvl": 571,
  "upgrade_level_legs": "4 / 8",
  "upgrade_level_shoulder": "1 / 8",
  "trinket_1_ilvl": 580,
  "blizzard_last_modified": {
    "$numberLong": "1725780009000"
  },
  "enchant_quality_feet": 0,
  "neck_ilvl": 580,
  "season_delves": 6,
  "best_neck_name": "Strapped Rescue-Keg",
  "week_heroic_dungeons": 8,
  "upgrade_level_waist": "4 / 8",
  "best_neck_id": 221060,
  "main_hand_quality": 3,
  "carved_crests": 135,
  "wrist_quality": 4,
  "finger_1_id": 215130,
  "rbg_rating": 0,
  "shoulder_quality": 3,
  "upgrade_level_feet": "4 / 8",
  "valorstones": 3194,
  "class": "Priest",
  "roster_visible": "yes",
  "finger_2_name": "Devout Zealot's Ring",
  "best_shoulder_ilvl": 571,
  "best_wrist_name": "Pioneer's Cloth Cuffs",
  "historical_great_vault_slot_7": "-",
  "back_quality": 3,
  "hands_id": 221067,
  "2v2_rating": 0,
  "rank": "Main",
  "tier_hands_ilvl": 0,
  "best_waist_ilvl": 580,
  "back_ilvl": 571,
  "best_trinket_1_quality": 3,
  "highest_ilvl_ever_equipped": 579.81,
  "gem_list": "2",
  "honorable_kills": 11180,
  "best_finger_2_quality": 3,
  "tier_head_ilvl": 0,
  "gilded_crests": 0,
  "upgrade_level_back": "1 / 8",
  "profession_2": "Tailoring (69)",
  "chest_ilvl": 580,
  "hands_name": "Punctured Apiary Gloves",
  "best_chest_id": 224666,
  "back_id": 224664,
  "season_highest_m+": 0,
  "best_main_hand_ilvl": 580,
  "best_legs_quality": 3,
  "trinket_1_quality": 3,
  "best_chest_name": "Alighted Robes of the Fireflash",
  "upgrade_level_wrist": "-",
  "best_back_quality": 3,
  "enchant_quality_legs": 0,
  "WCL_raid_finder": "-|-|-|-|-|-|-|-",
  "Ulgrax_The Bloodbound Horror_Sikran_Rasha'nan_Broodtwister Ovi'nax_Nexus-Princess Ky'veza_The Silken Court_Queen Ansurek@Nerub-ar Palace@": "0|0|0|0|0|0|0|0",
  "total_renown_score": 62054,
  "enchant_quality_waist": 0,
  "chest_quality": 3,
  "head_ilvl": 571,
  "legs_quality": 3,
  "city_of_threads_total": 0,
  "spark_off_hand_id": 0,
  "empty_sockets": 1,
  "head_name": "Intertwiner's Shadowcowl",
  "feet_quality": 3,
  "vault_visible": "yes",
  "legs_name": "Ancient Hardened Legwraps",
  "max_2v2_rating": 1500,
  "best_trinket_2_name": "Ara-Kara Sacbrood",
  "summary_visible": "yes",
  "dailies_done_total": 21455,
  "worldsoul_weekly": "no",
  "finger_1_name": "Malleable Band",
  "finger_1_quality": 4,
  "historical_great_vault_slot_5": "-",
  "wrist_ilvl": 590,
  "jewelry_sockets": 2,
  "historical_great_vault_slot_4": "-",
  "historical_great_vault_slot_2": "-",
  "best_finger_2_name": "Devout Zealot's Ring",
  "feet_ilvl": 580,
  "best_main_hand_id": 221085,
  "spark_off_hand_name": 0,
  "grim_batol_total": 0,
  "historical_great_vault_slot_8": "-",
  "best_neck_quality": 3,
  "best_feet_quality": 3,
  "finger_1_ilvl": 590,
  "finger_2_quality": 3,
  "raids_heroic": "0|0|0|0|0|0|0|0",
  "trinket_2_quality": 4,
  "cutting_edge": 4,
  "hallowfall_arathi_renown": 6,
  "best_back_ilvl": 571,
  "Alchemy_210816_Algari Alchemist Stone|Alchemy_212719_Algari Flask Cauldron|Alchemy_212751_Algari Potion Cauldron|Blacksmithing_222441_Everforged Warglaive|Blacksmithing_222443_Everforged Greataxe|Blacksmithing_222440_Everforged Longsword|Blacksmithing_222439_Everforged Dagger|Blacksmithing_222435_Everforged Vambraces|Blacksmithing_222436_Everforged Pauldrons|Blacksmithing_222430_Everforged Breastplate|Blacksmithing_222442_Everforged Mace|Blacksmithing_222432_Everforged Defender|Blacksmithing_222429_Everforged Sabatons|Blacksmithing_222431_Everforged Greatbelt|Blacksmithing_222438_Everforged Stabber|Blacksmithing_222433_Everforged Helm|Blacksmithing_222434_Everforged Legplates|Blacksmithing_222437_Everforged Gauntlets|Blacksmithing_222451_Charged Slicer|Blacksmithing_222447_Charged Claymore|Blacksmithing_222448_Charged Halberd|Blacksmithing_222446_Charged Facesmasher|Blacksmithing_222444_Charged Hexsword|Blacksmithing_222445_Charged Runeaxe|Blacksmithing_222449_Charged Crusher|Blacksmithing_222450_Charged Invoker|Blacksmithing_222459_Beledar's Bulwark|Blacksmithing_222458_Sanctified Steps|Blacksmithing_222463_Siphoning Stiletto|Blacksmithing_222493_Artisan Leatherworker's Toolset|Blacksmithing_222491_Artisan Needle Set|Blacksmithing_222492_Artisan Leatherworker's Knife|Blacksmithing_222488_Artisan Sickle|Blacksmithing_222489_Artisan Pickaxe|Blacksmithing_222490_Artisan Skinning Knife|Jewelcrafting_213777_Magnificent Jeweler's Setting|Jewelcrafting_213741_Culminating Blasphemite|Jewelcrafting_213744_Elusive Blasphemite|Jewelcrafting_213738_Insightful Blasphemite|Jewelcrafting_215133_Binding of Binding|Jewelcrafting_215134_Fractured Gemstone Locket|Jewelcrafting_215135_Ring of Earthen Craftsmanship|Jewelcrafting_215136_Amulet of Earthen Craftsmanship|Jewelcrafting_215125_Enchanter's Crystal|Jewelcrafting_215124_Extravagant Loupes|Jewelcrafting_215123_Forger's Font Inspector|Jewelcrafting_215122_Novelist's Specs|Inscription_222625_Transcribe to Radiance|Inscription_222624_Transcribe to Symbiosis|Inscription_222623_Transcribe to Vivacity|Inscription_222622_Transcribe to Ascension|Inscription_226025_Darkmoon Sigil: Radiance|Inscription_226031_Darkmoon Sigil: Vivacity|Inscription_226028_Darkmoon Sigil: Symbiosis|Inscription_226022_Darkmoon Sigil: Ascension|Inscription_222568_Vagabond's Bounding Baton|Inscription_222567_Vagabond's Careful Crutch|Inscription_222566_Vagabond's Torch|Inscription_222600_Contract: Assembly of the Deeps|Inscription_222603_Contract: Hallowfall Arathi|Inscription_222606_Contract: The Severed Threads|Inscription_222597_Contract: Council of Dornogal|Inscription_219660_Algari Missive of Crafting Speed|Inscription_222644_Algari Missive of Deftness|Inscription_222638_Algari Missive of Finesse|Inscription_222626_Algari Missive of Ingenuity|Inscription_222632_Algari Missive of Multicraft|Inscription_222641_Algari Missive of Perception|Inscription_222629_Algari Missive of Resourcefulness|Inscription_222578_Inscribed Rolling Pin|Inscription_222574_Silver Tongue's Quill|Inscription_222576_Patient Alchemist's Mixing Rod|Inscription_222548_Algari Treatise on Inscription|Inscription_222554_Algari Treatise on Blacksmithing|Inscription_222550_Algari Treatise on Enchanting|Inscription_222621_Algari Treatise on Engineering|Inscription_222552_Algari Treatise on Herbalism|Inscription_222546_Algari Treatise on Alchemy|Inscription_222551_Algari Treatise on Jewelcrafting|Inscription_222549_Algari Treatise on Leatherworking|Inscription_222553_Algari Treatise on Mining|Inscription_222649_Algari Treatise on Skinning|Inscription_222547_Algari Treatise on Tailoring|Inscription_219933_Algari Competitor's Emblem|Inscription_225368_Algari Competitor's Pillar|Inscription_225369_Algari Competitor's Staff|Inscription_225372_Algari Competitor's Lamp|Inscription_219932_Algari Competitor's Insignia of Alacrity|Inscription_219931_Algari Competitor's Medallion|Tailoring_222817_Consecrated Cloak|Tailoring_222816_Consecrated Cord|Tailoring_222815_Consecrated Cuffs|Tailoring_222822_Consecrated Gloves|Tailoring_222818_Consecrated Hood|Tailoring_222820_Consecrated Leggings|Tailoring_222821_Consecrated Mantle|Tailoring_222819_Consecrated Robe|Tailoring_222814_Consecrated Slippers|Tailoring_222850_Artisan Alchemist's Robe|Tailoring_222849_Artisan Enchanter's Hat|Tailoring_222851_Artisan Chef's Hat|Tailoring_222848_Artisan Fishing Cap|Tailoring_222847_Artisan Gardening Hat|Tailoring_222859_Concoctor's Clutch|Tailoring_222864_Darkmoon Duffle|Tailoring_222866_Excavator's Haversack|Tailoring_222860_Ignition Satchel|Tailoring_222867_Jeweler's Purse|Tailoring_222862_Magically Infinite Messenger|Tailoring_222863_Prodigy's Toolbox|Tailoring_224852_The Severed Satchel|Tailoring_222856_Duskweave Bag|Tailoring_222854_Dawnweave Reagent Bag|Tailoring_222865_Gardener's Seed Satchel|Tailoring_225936_Hideseeker's Tote|Tailoring_222861_Hideshaper's Workbag|Tailoring_222807_Grips of the Woven Dawn|Tailoring_222808_Treads of the Woven Dawn|Tailoring_222811_Warm Sunrise Bracers|Tailoring_222812_Cool Sunset Bracers|Tailoring_222809_Gloves of the Woven Dusk|Tailoring_222810_Slippers of the Woven Dusk|Tailoring_222868_Dawnthread Lining|Tailoring_222872_Duskthread Lining|Tailoring_222894_Daybreak Spellthread|Tailoring_222891_Sunset Spellthread|Tailoring_217120_Algari Competitor's Cloth Bands|Tailoring_217125_Algari Competitor's Cloth Cloak|Tailoring_217122_Algari Competitor's Cloth Gloves|Tailoring_217121_Algari Competitor's Cloth Hood|Tailoring_217117_Algari Competitor's Cloth Leggings|Tailoring_217124_Algari Competitor's Cloth Sash|Tailoring_217118_Algari Competitor's Cloth Shoulderpads|Tailoring_217119_Algari Competitor's Cloth Treads|Tailoring_217123_Algari Competitor's Cloth Tunic|Leatherworking_219906_Defender's Armor Kit|Leatherworking_219909_Stormbound Armor Kit|Leatherworking_219334_Rune-Branded Armbands|Leatherworking_219333_Rune-Branded Grasps|Leatherworking_219329_Rune-Branded Hood|Leatherworking_219327_Rune-Branded Kickers|Leatherworking_219332_Rune-Branded Legwraps|Leatherworking_219330_Rune-Branded Mantle|Leatherworking_219328_Rune-Branded Tunic|Leatherworking_219331_Rune-Branded Waistband|Leatherworking_219339_Glyph-Etched Binding|Leatherworking_219336_Glyph-Etched Breastplate|Leatherworking_219340_Glyph-Etched Cuisses|Leatherworking_219338_Glyph-Etched Epaulets|Leatherworking_219341_Glyph-Etched Gauntlets|Leatherworking_219337_Glyph-Etched Guise|Leatherworking_219335_Glyph-Etched Stompers|Leatherworking_219342_Glyph-Etched Vambraces|Leatherworking_219492_Sanctified Torchbearer's Grips|Leatherworking_219489_Waders of the Unifying Flame|Leatherworking_219495_Blessed Weapon Grip|Leatherworking_219502_Adrenal Surge Clasp|Leatherworking_219501_Vambraces of Deepening Darkness|Leatherworking_219504_Writhing Armor Banding|Leatherworking_219509_Busy Bee's Buckle|Leatherworking_219513_Roiling Thunderstrike Talons|Leatherworking_219511_Rook Feather Wristwraps|Leatherworking_219508_Reinforced Setae Flyers|Leatherworking_219507_Smoldering Pollen Hauberk|Leatherworking_219512_Weathered Stormfront Vest|Leatherworking_217130_Algari Competitor's Leather Belt|Leatherworking_217126_Algari Competitor's Leather Boots|Leatherworking_217127_Algari Competitor's Leather Chestpiece|Leatherworking_217132_Algari Competitor's Leather Gloves|Leatherworking_217128_Algari Competitor's Leather Mask|Leatherworking_217129_Algari Competitor's Leather Shoulderpads|Leatherworking_217131_Algari Competitor's Leather Trousers|Leatherworking_217133_Algari Competitor's Leather Wristwraps|Leatherworking_217135_Algari Competitor's Chain Chainmail|Leatherworking_217136_Algari Competitor's Chain Cowl|Leatherworking_217141_Algari Competitor's Chain Cuffs|Leatherworking_217137_Algari Competitor's Chain Epaulets|Leatherworking_217140_Algari Competitor's Chain Gauntlets|Leatherworking_217138_Algari Competitor's Chain Girdle|Leatherworking_217139_Algari Competitor's Chain Leggings|Leatherworking_217134_Algari Competitor's Chain Treads|Leatherworking_219871_Arathi Leatherworker's Smock|Leatherworking_219870_Charged Scrapmaster's Gauntlets|Leatherworking_219874_Earthen Forgemaster's Apron|Leatherworking_219876_Earthen Jeweler's Cover|Leatherworking_219867_Stonebound Herbalist's Pack|Leatherworking_219869_Deep Tracker's Cap|Leatherworking_219868_Deep Tracker's Pack|Leatherworking_219872_Nerubian Alchemist's Hat|Enchanting_224073_Enchanted Gilded Harbinger Crest|Enchanting_224072_Enchanted Runed Harbinger Crest|Enchanting_224069_Enchanted Weathered Harbinger Crest|Enchanting_224173_Concentration Concentrate|Enchanting_224108_Oil of Beledar's Grace|Enchanting_224111_Oil of Deep Toxins|Enchanting_224405_Scepter of Radiant Magics|Enchanting_224116_Runed Null Stone Rod|Enchanting_224115_Runed Ironclaw Rod|Enchanting_50475_Cavalry's March|Enchanting_50477_Defender's March|Enchanting_50476_Scout's March|Enchanting_50521_Chant of Armored Avoidance|Enchanting_50523_Chant of Armored Leech|Enchanting_50525_Chant of Armored Speed|Enchanting_50503_Council's Intellect|Enchanting_50504_Crystalline Radiance|Enchanting_50505_Oathsworn's Strength|Enchanting_50501_Stormrider's Agility|Enchanting_50531_Chant of Burrowing Rapidity|Enchanting_50529_Chant of Leeching Fangs|Enchanting_50527_Chant of Winged Grace|Enchanting_50506_Algari Deftness|Enchanting_50507_Algari Finesse|Enchanting_50508_Algari Ingenuity|Enchanting_50509_Algari Perception|Enchanting_50510_Algari Resourcefulness|Enchanting_50479_Radiant Critical Strike|Enchanting_50481_Radiant Haste|Enchanting_50483_Radiant Mastery|Enchanting_50485_Radiant Versatility|Enchanting_50532_Cursed Critical Strike|Enchanting_50533_Cursed Haste|Enchanting_50534_Cursed Mastery|Enchanting_50535_Cursed Versatility|Enchanting_50486_Authority of Air|Enchanting_50487_Authority of Fiery Resolve|Enchanting_50488_Authority of Radiant Power|Enchanting_50511_Authority of Storms|Enchanting_50536_Authority of the Depths|Engineering_221868_Entropy Enhancer|Engineering_221865_Chaos Circuit|Engineering_225987_Bottled Brilliance|Engineering_221862_Safety Switch|Engineering_221859_Gyrating Gear|Engineering_221856_Whimsical Wiring|Engineering_221853_Handful of Bismuth Bolts|Engineering_221926_Blame Redirection Device|Engineering_221932_Complicated Fuse Box|Engineering_221923_Recalibrated Safety Switch|Engineering_221938_Concealed Chaos Module|Engineering_221941_Energy Redistribution Beacon|Engineering_221935_Pouch of Pocket Grenades|Engineering_221920_Adjustable Cogwheel|Engineering_221917_Impeccable Cogwheel|Engineering_221914_Overclocked Cogwheel|Engineering_221911_Serrated Cogwheel|Engineering_221805_Blasting Bracers|Engineering_221808_Clanking Cuffs|Engineering_221804_Dangerous Distraction Inhibitor|Engineering_221802_Overclocked Idea Generator|Engineering_221969_P.0.W. x2|Engineering_221801_Studious Brilliance Expeditor|Engineering_221803_Supercharged Thought Enhancer|Engineering_221806_Venting Vambraces|Engineering_221807_Whirring Wristwraps|Engineering_221789_Aqirite Brainwave Projector|Engineering_221791_Aqirite Fisherfriend|Engineering_221798_Aqirite Fueled Samophlange|Engineering_221796_Aqirite Miner's Headgear|Engineering_221793_Lapidary's Aqirite Clamps|Engineering_221800_Miner's Aqirite Hoard|Engineering_221787_Spring-Loaded Aqirite Fabric Cutters|Engineering_221788_Bismuth Brainwave Projector|Engineering_221790_Bismuth Fisherfriend|Engineering_221795_Bismuth Miner's Headgear|Engineering_221792_Lapidary's Bismuth Clamps|Engineering_221799_Miner's Bismuth Hoard|Engineering_221786_Spring-Loaded Bismuth Fabric Cutters|Engineering_224586_Box o' Booms|Engineering_221880_Potion Bomb of Power|Engineering_221876_Potion Bomb of Recovery|Engineering_221872_Potion Bomb of Speed|Engineering_221967_Crowd Pummeler 2-30|Engineering_221953_Convincingly Realistic Jumper Cables|Engineering_221945_Irresistible Red Button|Engineering_221949_Pausing Pylon|Engineering_221957_Algari Repair Bot 11O|Engineering_221959_Portable Profession Possibility Projector|Engineering_219387_Barrel of Fireworks|Engineering_221962_Defective Escape Pod|Engineering_221964_Filmless Camera|Engineering_221966_Wormhole Generator: Khaz Algar|Engineering_219403_Stonebound Lantern|Engineering_221904_Tinker: Earthen Delivery Drill|Engineering_221908_Tinker: Heartseeking Health Injector|Engineering_217155_Algari Competitor's Cloth Bracers|Engineering_217151_Algari Competitor's Cloth Goggles|Engineering_217156_Algari Competitor's Leather Bracers|Engineering_217152_Algari Competitor's Leather Goggles|Engineering_217157_Algari Competitor's Mail Bracers|Engineering_217153_Algari Competitor's Mail Goggles|Engineering_217158_Algari Competitor's Plate Bracers|Engineering_217154_Algari Competitor's Plate Goggles|Engineering_225370_Algari Competitor's Rifle": "|50479|224069|224072|224073|217117|",
  "best_back_id": 224664,
  "best_chest_ilvl": 580,
  "WCL_heroic": "-|-|-|-|-|-|-|-",
  "best_feet_name": "Cloudstrider Soles",
  "raids_raid_finder": "0|0|0|0|0|0|0|0",
  "head_enchant": 0,
  "ansurek_mount": "no",
  "best_off_hand_ilvl": 0,
  "weekly_event_completed": "no",
  "3v3_season_played": 0,
  "enchant_quality_finger_2": 2,
  "raids_mythic": "0|0|0|0|0|0|0|0",
  "role": "Ranged",
  "dungeons_done_total": 0,
  "best_neck_ilvl": 580,
  "the_dawnbreaker_total": 0,
  "grim_batol_score": 0,
  "best_chest_quality": 3,
  "best_wrist_quality": 4,
  "best_main_hand_quality": 3,
  "overview_visible": "yes",
  "waist_name": "Alighted Cord of the Feverflare"
}